In [1]:
!pip install openai -q
!pip install llama-index -q
!pip install sentence-transformers -q
!pip install llama-index-llms-openai -q
!pip install llama-index-embeddings-openai -q
!pip install llama-index-finetuning -q
!pip install llama-index-embeddings-openai -q

In [2]:
!pip install llama-index-embeddings-huggingface -q

In [3]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import json
import openai
import torch
import os
import shutil
import sentence_transformers
from tqdm.notebook import tqdm
from llama_index.core import SimpleDirectoryReader, ServiceContext, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import MetadataMode, TextNode
from llama_index.finetuning import (
    generate_qa_embedding_pairs, EmbeddingQAFinetuneDataset, EmbeddingAdapterFinetuneEngine, SentenceTransformersFinetuneEngine)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.embeddings.utils import resolve_embed_model
from llama_index.embeddings.adapter.utils import TwoLayerNN
from llama_index.finetuning import EmbeddingQAFinetuneDataset

In [4]:
data = pd.read_csv('data/train_dataset_Датасет.csv',sep=',')


In [5]:

train_questions = data['QUESTION'].tolist()
train_answers = data['ANSWER'].tolist()


In [6]:
queries = {
    str(i): question for i, question in enumerate(train_questions)
}

In [7]:
corpus = {
    str(i): answer for i, answer in enumerate(train_answers)
}

In [8]:
relevant_docs = {
    str(i): [str(i)] for i in range(len(train_questions))
}

In [9]:
train_dataset = EmbeddingQAFinetuneDataset(
    queries=queries,
    corpus=corpus,
    relevant_docs=relevant_docs
)

In [10]:
finetune_engine = SentenceTransformersFinetuneEngine(
    dataset=train_dataset,
    model_id="sentence-transformers/LaBSE",
    model_output_path="finetuned_labse",
    epochs=5
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
finetune_engine.finetune()


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/274 [00:00<?, ?it/s]

Iteration:   0%|          | 0/274 [00:00<?, ?it/s]

Iteration:   0%|          | 0/274 [00:00<?, ?it/s]

Iteration:   0%|          | 0/274 [00:00<?, ?it/s]

Iteration:   0%|          | 0/274 [00:00<?, ?it/s]

In [12]:
embed_model_02 = finetune_engine.get_finetuned_model()


In [13]:
 embed_model_02

HuggingFaceEmbedding(model_name='finetuned_labse', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7d88cd5b06d0>, num_workers=None, max_length=256, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None)

In [31]:
test_data = pd.read_excel('data/test_dataset.xlsx')
test_questions =  test_data['QUESTION_2'].tolist()
test_answers = test_data['ANSWER'].tolist()


In [32]:
embed_model = embed_model_02

In [33]:
def get_embeddings(texts, model):
    return model._get_text_embeddings(texts)

In [36]:
test_question_embeddings = get_embeddings(test_questions, embed_model_02)
test_answer_embeddings = get_embeddings(test_answers, embed_model_02)

similarities = cosine_similarity(test_question_embeddings, test_answer_embeddings)

top_k = 3
top_k_indices = np.argsort(-similarities, axis=1)[:, :top_k]

true_labels = np.arange(len(test_questions))

predicted_labels_top_k = np.zeros_like(true_labels)
for i, true_label in enumerate(true_labels):
    if true_label in top_k_indices[i]:
        predicted_labels_top_k[i] = true_label

precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels_top_k, average='macro', zero_division=0)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

Precision: 0.84
Recall: 0.8666666666666667
F1-Score: 0.8444444444444446


In [38]:
import torch

torch.save(embed_model_02._model.state_dict(), "finetuned_labse_model.pth")